# Tensorflow
Sean Wade

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [7]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

y = tf.nn.softmax(tf.matmul(x,W) + b)
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())
for i in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})
    
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

Exception AssertionError: AssertionError("Nesting violated for default stack of <type 'weakref'> objects",) in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x10237d2d0>> ignored


0.9202


___

In [3]:
def weight_variable(shape, n=""):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name=n)

def bias_variable(shape, n=""):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name=n)

In [4]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [5]:
x = tf.placeholder(tf.float32, shape=[None, 784], name="x")
y_ = tf.placeholder(tf.float32, shape=[None, 10], name="y_")

with tf.name_scope("Wx_b") as scope:
    
    #with tf.name_scope("layer_1") as sub_scope1:
    W_conv1 = weight_variable([5, 5, 1, 32], n="W_conv1")
    b_conv1 = bias_variable([32], n="b_conv1")
    x_image = tf.reshape(x, [-1,28,28,1])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)

    #with tf.name_scope("layer_2") as sub_scope2:
    W_conv2 = weight_variable([5, 5, 32, 64], n="W_conv2")
    b_conv2 = bias_variable([64], n="b_conv2")

    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)

    #with tf.name_scope("densely_connected_layer") as sub_scope3:
    W_fc1 = weight_variable([7 * 7 * 64, 1024], n="W_fc1")
    b_fc1 = bias_variable([1024], n="b_fc")

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

    #with tf.name_scope("dropout") as sub_scope4:
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    #with tf.name_scope("readout_layer") as sub_scope5:
    W_fc2 = weight_variable([1024, 10], n="W_fc2")
    b_fc2 = bias_variable([10], n="b_fc2")

    y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)


In [6]:
with tf.name_scope("cost_function") as scope:
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
    
with tf.name_scope("optimizer") as scope:
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

with tf.name_scope("accuracy") as scope:
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

wc1_hist = tf.histogram_summary('W_conv1', W_conv1)
wc2_hist = tf.histogram_summary('W_conv2', W_conv2)
accuarcy_summary = tf.scalar_summary("accuracy", accuracy)
cross_entropy_summary = tf.scalar_summary("loss", cross_entropy)

sess = tf.InteractiveSession()     
sess.run(tf.initialize_all_variables())
summary_writer = tf.train.SummaryWriter( "./tf_logs", graph=sess.graph )
merged = tf.merge_all_summaries()

for i in range(1500):
    batch = mnist.train.next_batch(50)
    if i%10 == 0:
        train_accuracy = accuracy.eval(feed_dict={
                x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
        #summary, _ = train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
        summary, _, acc = sess.run([merged, train_step, accuracy], feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
        summary_writer.add_summary(summary, i)
    else:
        sess.run([train_step], feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

        
    
print "-----COMPLETE-----"
#print("test accuracy %g"%accuracy.eval(feed_dict={
#    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))#

step 0, training accuracy 0.12
step 10, training accuracy 0.24
step 20, training accuracy 0.36
step 30, training accuracy 0.6
step 40, training accuracy 0.6
step 50, training accuracy 0.76
step 60, training accuracy 0.64
step 70, training accuracy 0.82
step 80, training accuracy 0.84
step 90, training accuracy 0.84
step 100, training accuracy 0.8
step 110, training accuracy 0.86
step 120, training accuracy 0.92
step 130, training accuracy 0.9
step 140, training accuracy 0.9
step 150, training accuracy 0.86
step 160, training accuracy 0.8
step 170, training accuracy 0.92
step 180, training accuracy 0.92
step 190, training accuracy 0.8
step 200, training accuracy 0.94
step 210, training accuracy 0.88
step 220, training accuracy 0.82
step 230, training accuracy 0.94
step 240, training accuracy 0.92
step 250, training accuracy 0.86
step 260, training accuracy 0.9
step 270, training accuracy 0.92
step 280, training accuracy 0.94
step 290, training accuracy 0.92
step 300, training accuracy 0

<img  src="Screen Shot 2016-09-21 at 7.02.39 PM.png"/>

<img  src="Screen Shot 2016-09-21 at 7.02.51 PM.png"/>

<img  src="Screen Shot 2016-09-21 at 7.05.59 PM.png"/>

<img  src="Screen Shot 2016-09-21 at 7.06.23 PM.png"/>